In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
import matplotlib
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import librosa
import scipy
from IPython.display import Audio

ModuleNotFoundError: No module named 'xgboost'

In [44]:
# load the model from disk
rf_classifier = pickle.load(open('models_saved/rf_classifier_model.pkl', 'rb'))
lr_classifier = pickle.load(open('models_saved/lr_classifier_model.pkl', 'rb'))
xgb_classifier = pickle.load(open('models_saved/xgb_classifier_model.pkl', 'rb'))
svm_classifier = pickle.load(open('models_saved/rf_classifier.pkl', 'rb'))

label_dict = { 0:'Hip-Hop',
               1:'Pop',
               2:'Reggae',
               3:'Mbalax'
             }


NameError: name 'pickle' is not defined

In [29]:
# Predict function which use xgboost model, you can change it depending on the model want to use

def GenrePrediction(file_Location,filename):
    y, sr = librosa.load(file_Location+"/"+filename, sr = 22050) # Use the default sampling rate of 22,050 Hz
    
    ######################
    # Feature extraction #
    ######################
    feature_list = [filename]

    feature_list.append(np.mean(abs(y)))
    feature_list.append(np.std(y))
    feature_list.append(scipy.stats.skew(abs(y)))
    feature_list.append(scipy.stats.kurtosis(y))

    zcr = librosa.feature.zero_crossing_rate(y + 0.0001, frame_length=2048, hop_length=512)[0]
    feature_list.append(np.mean(zcr))
    feature_list.append(np.std(zcr))

    rmse = librosa.feature.rmse(y + 0.0001)[0]
    feature_list.append(np.mean(rmse))
    feature_list.append(np.std(rmse))

    tempo = librosa.beat.tempo(y, sr=sr)
    feature_list.extend(tempo)

    spectral_centroids = librosa.feature.spectral_centroid(y+0.01, sr=sr)[0]
    feature_list.append(np.mean(spectral_centroids))
    feature_list.append(np.std(spectral_centroids))

    spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(y+0.01, sr=sr, p=2)[0]
    spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(y+0.01, sr=sr, p=3)[0]
    spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(y+0.01, sr=sr, p=4)[0]
    feature_list.append(np.mean(spectral_bandwidth_2))
    feature_list.append(np.std(spectral_bandwidth_2))
    feature_list.append(np.mean(spectral_bandwidth_3))
    feature_list.append(np.std(spectral_bandwidth_3))
    feature_list.append(np.mean(spectral_bandwidth_3))
    feature_list.append(np.std(spectral_bandwidth_3))

    spectral_contrast = librosa.feature.spectral_contrast(y, sr=sr, n_bands = 6, fmin = 200.0)
    feature_list.extend(np.mean(spectral_contrast, axis=1))
    feature_list.extend(np.std(spectral_contrast, axis=1))

    spectral_rolloff = librosa.feature.spectral_rolloff(y+0.01, sr=sr, roll_percent = 0.85)[0]
    feature_list.append(np.mean(spectral_rolloff))
    feature_list.append(np.std(spectral_rolloff))

    mfccs = librosa.feature.mfcc(y, sr=sr, n_mfcc=20)
    feature_list.extend(np.mean(mfccs, axis=1))
    feature_list.extend(np.std(mfccs, axis=1))

    chroma_stft = librosa.feature.chroma_stft(y, sr=sr, hop_length=1024)
    feature_list.extend(np.mean(chroma_stft, axis=1))
    feature_list.extend(np.std(chroma_stft, axis=1))

    feature_list[1:] = np.round(feature_list[1:], decimals=3)
    
    predicted=xgb_classifier.predict(feature_list[1:])
    
    print("C'est du "+label_dict[int(predicted)])
    
    return Audio(y, rate = sr)


In [59]:
GenrePrediction('wav_files','son_test.wav') 

C'est du Mbalax
